In [1]:
import numpy as np
from torch.autograd import Variable
import torch
from torch import nn, autograd
import torch.nn.functional as F

# Experiment One
$$x \to Bernulli(0.5)$$ 
$$y|x = 0 \to Bernulli(0.9)$$ 
$$y|x = 1 \to Bernulli(0.3)$$

we use a linear-> sigmoid regression to learn the conditional probability

In [36]:
# parameters and model
sig = nn.Sigmoid()
W = Variable(torch.rand(1), requires_grad=True)
b = Variable(torch.rand(1), requires_grad=True)
def net(x):
    y_ = W*x + b
    y_ = sig(y_)
    loss = -(y*torch.log(y_) + (1-y)*torch.log(1-y_)) 
    return y_, loss

def gen_data():
    x = Variable(torch.bernoulli(torch.ones(1)*0.5))
    if x.data.numpy() == 0:
        y = Variable(torch.bernoulli(torch.ones(1)*0.9))
    else:
        y = Variable(torch.bernoulli(torch.ones(1)*0.3))
    return x,y

In [84]:
learning_rate = 0.05
for t in range(1,2000):
    x,y = gen_data()
    y_, loss = net(x)
    loss.backward()
    b.data -= learning_rate * b.grad.data
    b.grad.data.zero_()
    W.data -= learning_rate * W.grad.data
    W.grad.data.zero_()

In [85]:
# test
x = Variable(torch.ones(1))
y_,_ = net(x)
print('True para is 0.3, the perdiction is {}'.format(y_.data.numpy()))

x = Variable(torch.ones(1)*0)
y_,_ = net(x)
print('True para is 0.9, the perdiction is {}'.format(y_.data.numpy()))

True para is 0.3, the perdiction is [ 0.29190451]
True para is 0.9, the perdiction is [ 0.91689456]


# More compact

In [56]:
D_in, D_out = 1, 1
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_out),
    torch.nn.Sigmoid()
)
loss_fn = torch.nn.BCELoss()
learning_rate = 0.02
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(2000):
    x,y = gen_data()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [57]:
# test
x = Variable(torch.ones(1))
y_ = model(x)
print('True para is 0.3, the perdiction is {}'.format(y_.data.numpy()))

x = Variable(torch.ones(1)*0)
y_ = model(x)
print('True para is 0.9, the perdiction is {}'.format(y_.data.numpy()))

True para is 0.3, the perdiction is [ 0.34832135]
True para is 0.9, the perdiction is [ 0.90798932]


# Batch Training

In [82]:
N, D_in, D_out =50, 1, 1
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_out),
    torch.nn.Sigmoid()
)
loss_fn = torch.nn.BCELoss()
learning_rate = 0.02
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(2000):
    data = [gen_data() for _ in range(N)]
    x = torch.cat([i for i,_ in data]).view(N, D_in)
    y = torch.cat([j for _,j in data]).view(N, D_out)
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    #if t%100 == 0:
    #    print(t, loss.data[0])
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [83]:
# test
x = Variable(torch.ones(1))
y_ = model(x)
print('True para is 0.3, the perdiction is {}'.format(y_.data.numpy()))

x = Variable(torch.ones(1)*0)
y_ = model(x)
print('True para is 0.9, the perdiction is {}'.format(y_.data.numpy()))

True para is 0.3, the perdiction is [ 0.2888166]
True para is 0.9, the perdiction is [ 0.90217209]


# Embeding learning

In [31]:
# embed 0,1 into vectors V[0],V[1]
im_D = 10
h_D = 50
V = torch.randn(2,10)
print('0_imbd is \n {} \n1_imbd is \n {}'.format(V[0].numpy(), V[1].numpy()))
model = torch.nn.Sequential(
    torch.nn.Linear(im_D, h_D),
    torch.nn.ReLU(),
    torch.nn.Linear(h_D, im_D))

def tran():
    

0_imbd is 
 [ 0.944502   -0.3706497  -1.83810687  0.23389538  1.62372994  0.39470863
 -2.22684503  1.26098907  2.35926294 -0.31866524] 
1_imbd is 
 [ 0.27534598  0.17846633  0.22618195 -0.24257904 -0.77979839  0.3432427
  0.41293901  1.77549076  0.92081589  0.19707948]
